#### Getting the data

In [1]:
import pandas as pd

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

#### Q1. Getting the embeddings model

In [4]:
model_name = 'multi-qa-mpnet-base-dot-v1'

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [5]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [6]:
answer_llm_emb = embedding_model.encode(answer_llm).tolist()
answer_llm_emb[0]

-0.4224465489387512

#### Q2. Computing the dot product

In [7]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [8]:
from tqdm.auto import tqdm

In [9]:
results_gpt4o = df.to_dict(orient='records')

In [10]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    return v_llm.dot(v_orig)

In [11]:
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity(record)
    similarity.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [12]:
df['cosine'] = similarity
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine, dtype: float64

31.674309

In [13]:
import pickle

In [14]:
with open("similarity_2507", "wb") as fp:   #Pickling
    pickle.dump(similarity, fp)

In [25]:
#with open("evaluations_test", "rb") as fp:   # Unpickling
    #b = pickle.load(fp)

#### Q3. Computing the cosine

In [15]:
import numpy as np

In [16]:
def compute_norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [17]:
def compute_similarity_norm(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    v_llm = compute_norm(embedding_model.encode(answer_llm))
    v_orig = compute_norm(embedding_model.encode(answer_orig))
    return v_llm.dot(v_orig)

In [18]:
similarity_norm = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity_norm(record)
    similarity_norm.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [19]:
with open("similarity_norm_2507", "wb") as fp:   #Pickling
    pickle.dump(similarity_norm, fp)

In [20]:
df['cosine_norm'] = similarity_norm
df['cosine_norm'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine_norm, dtype: float64

0.836235

#### Q4. Rouge

In [53]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [22]:
from rouge import Rouge
rouge_scorer = Rouge()

In [23]:
scores_list = []

for r in tqdm(results_gpt4o):
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    scores_list.append(scores)

  0%|          | 0/300 [00:00<?, ?it/s]

In [25]:
scores_list[10]

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

0.45454545454545453

#### Q5. Average rouge score

In [29]:
average_rouge = (0.45454545454545453+0.21621621621621623+0.3939393939393939)/3
average_rouge

0.35490035490035493

#### Q6. Average rouge score for all the data points

In [62]:
df_rouge = pd.DataFrame(columns=['rouge-1','rouge-2','rouge-l','rouge-avg'])

In [68]:
data = []

for i in range(len(scores_list)):
    rouge_1 = scores_list[i]['rouge-1']['f']
    rouge_2 = scores_list[i]['rouge-2']['f']
    rouge_l = scores_list[i]['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
    data.append({
        'rouge-1': rouge_1,
        'rouge-2': rouge_2,
        'rouge-l': rouge_l,
        'rouge-avg': rouge_avg
    })

In [69]:
df_rouge = pd.concat([df_rouge, pd.DataFrame(data)], ignore_index=True)

/tmp/ipykernel_2798/1645964286.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_rouge = pd.concat([df_rouge, pd.DataFrame(data)], ignore_index=True)


In [71]:
scores_list[0]

{'rouge-1': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524},
 'rouge-2': {'r': 0.017543859649122806,
  'p': 0.07142857142857142,
  'f': 0.028169010918468917},
 'rouge-l': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524}}

In [70]:
df_rouge

,rouge-1,rouge-2,rouge-l,rouge-avg
0,0.095238,0.028169,0.095238,0.072882
1,0.125000,0.055556,0.093750,0.091435
2,0.415584,0.177778,0.389610,0.327658
3,0.216216,0.047059,0.189189,0.150821
4,0.142076,0.033898,0.120219,0.098731
...,...,...,...,...
295,0.654545,0.540984,0.618182,0.604570
296,0.590164,0.460432,0.557377,0.535991
297,0.654867,0.564516,0.637168,0.618851
298,0.304762,0.132231,0.304762,0.247252


In [73]:
df_rouge.loc[:, 'rouge-2'].mean()

0.20696501983423318

In [21]:
scores_list = []

for r in tqdm(results_gpt4o):
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    scores_list.append(scores)

0.6506573240979582

In [29]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents_ml, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

#### Q4. Hit-rate for our search engine

In [8]:
from tqdm.auto import tqdm
import pandas as pd

In [2]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
def question_vector(question):
    v_q = embedding_model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [30]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_vector(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [26]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [32]:
hit_rate(relevance_total)

0.9398907103825137

#### Q5. Indexing with Elasticsearch

In [1]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200', timeout=600) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qt_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_26419/2728056219.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_client = Elasticsearch('http://localhost:9200', timeout=600)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'course-questions'})

In [5]:
doc_try = documents_ml[:2]

In [6]:
doc_try

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?'

In [9]:
for doc in tqdm(doc_try):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def elastic_search_knn(field, vector):
    
    knn_query = {
    "field": "qt_vector",
    "query_vector": vector,
    "k": 5,
    "num_candidates": 10000
    }

    search_query = {
        "knn": knn_query,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def question_text_vector_knn(question):
    v_q = embedding_model.encode(question)
    return elastic_search_knn('question_text_vector', v_q)

In [ ]:
question_text_vector_knn(user_question)

Highest score id: ee58a693

#### Q6. Hit-rate for Elasticsearch

In [ ]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_text_vector_knn(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
hit_rate(relevance_total)

0.9398907103825137